In this document, we are going to be pulling all of the stand up comedy transcripts from scrapsoftheloft.com. We will then write all of those transcripts to their own csv file.

In [ ]:
import requests
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
from bs4.element import Comment
import pandas as pd
from pprint import pprint
import re
import numpy as np

In [ ]:
#https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup

These chunks get all of the links from the page that houses links to transcripts and makes sure they aren't duplicates. 

In [ ]:
source = requests.get('https://scrapsfromtheloft.com/stand-up-comedy-scripts/').text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())

In [ ]:
transcripts = []
for a in soup.find_all("a", href=True):
    if "transcript" in a['href']:
        transcripts.append(a['href'])
#print(transcripts)
del transcripts[0:9]

This removes duplicate links, on the off chance that there are any. 

In [ ]:
tranSet = set(transcripts)
tranList = list(tranSet)
#print(tranList)

This chunk separates the links that aren't transcripts from the rest. 

In [ ]:
tranList = [item for item in tranList if "comedy/" in item]
#tranList

Now we are acquiring the transcripts 

In [ ]:
titles = []
Transcripts = dict()
for item in tranList:
    source = requests.get(item).text
    soup = BeautifulSoup(source, 'lxml')
    tranText = []    
    elements =  soup.find_all('div', attrs={'class':"elementor-widget-container"})
    for element in elements:
        para = element.find_all("p")
        if element.text.lower() == "share this article":
            break
        tranText.extend([item.text for item in para])
    url = item
    junk,title = url.split("comedy/")


    titles.append(title)
    Transcripts[item] = tranText



This chunk writes the individual transcripts txt files titles

In [ ]:
csv = []
for key in Transcripts:
    url = key
    junk, title = url.split("comedy/")
    #remove trailing forwardslash
    title = title[:-1]
    title = title + ".txt"
    csv.append(title)

Now we write the csv so we can load that into our next step, cleaning the data and get the descriptive statistics. 

In [ ]:
df = pd.DataFrame({'title': titles,
                   'link': tranList,
                   'CSV': csv})

In [ ]:
def year_from_title(title_series):
    # Regex search for 4 digits in a row
    years = title_series.apply(lambda x: re.findall(r"(\d{4})", x))
    series_dict = years.to_dict()
    for key in series_dict:
        if not series_dict[key]:
            series_dict[key] = np.nan
        else:
            series_dict[key] = series_dict[key][0]
    return pd.Series(series_dict)
            
df['year'] = year_from_title(df.title)

In [ ]:
def clean_titles(title_series):
    # Remove everything between parentheses (year)
    title_series = title_series.apply(lambda x: re.sub('\(.*?\)', '', x))
    # Remove all other extraneous text from titles
    remove_list = ['-','/','Transcript', 'Full', '[', ']']
    for i in remove_list:
        title_series = title_series.apply(lambda x: x.title().replace(i, " "))
    return title_series

df.title = clean_titles(df.title)

In [ ]:
#df.head(4)

In [ ]:
print(f'Length : {len(df)}')
df.to_csv("DF_TO_LOAD.csv", index = False)
df.head()